# Getting started

The following code block will attempt install all of the necessary packages required for 
these exercises. We assume that any neuroimaging tools required are already installed or
Docker or Singularity is avaialble for use.

In [ ]:
!pip install -q \
  'nibabel' \
  'niwrap' \
  'nilearn[plotting]'

`Niwrap` has a number of a "Runners" to for executing neuroimaging tools - select the 
runner of choice to additional steps for setting up the runner!

1. Local installations (**default** - nothing needs to be done)
2. [DockerRunner](#Docker-Runner)
3. [SingularityRunner](#Singularity-Runner)

## Runner Setup

### Local Runner

You don't need to do anything - this is the default runner that is used. It will attempt to use installations found on the system.

### Docker Runner 
_**Skip this if using local installation or Singularity**_

While downloading containers when using the Docker runner isn't strictly necessary (containers are pulled automatically by the runner), we will do so here before attempting 
any exercises to minimize having to wait for containers to be pulled later on.

In [ ]:
%%bash

docker pull antsx/ants:v2.5.3 -q
# docker pull brainlife/fsl:6.0.4-patched2 -q # Uncomment to download large (7GB) container

We will also set `niwrap` to use the DockerRunner!

In [ ]:
import niwrap

niwrap.use_docker()

### Singularity Runner 
_**Skip this if using local installation or Docker**_

Downloading containers is required when using the Singularity runner. Additionally,
we will need to tell our runner where we can find these containers after they have been
downloaded!

In [ ]:
%%bash

mkdir -p ../containers
singularity pull ../containers/fsl_6.0.4-patched2.sif docker://brainlife/fsl:6.0.4-patched2
singularity pull ../containers/ants_v2.5.3.sif docker://antsx/ants:v2.5.3

In [ ]:
from pathlib import Path

import niwrap

niwrap.use_singularity(
    images={
        "brainlife/fsl:6.0.4-patched2": Path("../containers/fsl_6.0.4-patched2.sif").resolve()
    }
)

## Exercises 

In the following exercises, we will use a combination of commonly used neuroimaging 
Python libraries and tools to perform a number of different processing tasks. Let's 
first take a look at what our data folder looks like!

In [ ]:
!tree `realpath ../data`

As you can see, we will be using some MNI template data in different spaces.

### Example

Before we actually start doing things, lets take a look at some features of `niwrap`. 
All supported neuroimaging tools are available as a sub-module of `niwrap` - that is
we can follow the `from niwrap import ...` syntax!

Let's try and load in the `FSL` sub-module and figure out what commands are available! 

In [ ]:
from niwrap import fsl

In [ ]:
# Autocomplete is available!
fsl.

What if we want to execute a command let `bet` for brain extraction, but can't remember
the parameters?

In [ ]:
# All of the docstrings have also been extracted and made available, along with type hints!
help(fsl.bet)

In this example, we will use [FSL](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/) to perform brain extraction on the T1w image in MNI152NLin2009cAsym space.

In [ ]:
# We'll first grab our file
from pathlib import Path

data_dir = Path("../data").resolve()
if not data_dir.exists():
    raise FileNotFoundError("Data directory could not be found.")

mni2009c = data_dir / "tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz"

In [ ]:
# Start with importing fsl
from niwrap import fsl

'''
bet ../data/tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz out -f 0.7
'''

For some tools, a default output file name is already provided. You can also overwrite the output file name (we'll see this in a later example). Let's visualize what this looks like!

In [ ]:
from nilearn import plotting

plotting.plot_anat(
    ???,
    title="Brain extracted T1w"
)
plotting.show()

### Exercise 1

For our first task, we will perform a slightly more complicated task using [ANTs](https://github.com/ANTsX/ANTs) to transform `tpl-MNI152NLin6Asym` to `MNI152NLin2009cAsym`. Again, all of the necessary data to do this is already available!

In [ ]:
# We'll first get grab all our input variables
from pathlib import Path

data_dir = Path("../data").resolve()
if not data_dir.exists():
    raise FileNotFoundError("Data directory could not be found.")

t1w = data_dir / "tpl-MNI152NLin6Asym_res-02_T1w.nii.gz"
mask = data_dir / "tpl-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz"
xfm = data_dir / "tpl-MNI152NLin6Asym_from-MNI152NLin2009cAsym_mode-image_xfm.mat"
ref = data_dir / "tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz"

In [ ]:
# Start by trying to import ants
from niwrap import ants 

We know we need the command `antsApplyTransforms`, but we don't know what equivalent call
is in `niwrap` nor the parameters that need to provided.

In [ ]:
# Try the autocomplete feature, as well as printing out the help
help(ants.ants_apply_transforms)

Let's start by applying our transformation to the T1w.

In [ ]:
# Note this uses what is known as 'sub-commands' in niwrap
'''
antsApplyTransforms \
  --dimensionality 3 \
  --input-image-type 0 \
  --input ../data/tpl-MNI152NLin6Asym_res-02_T1w.nii.gz \
  --reference-image ../data/tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz \
  --output tpl-MNI152NLin6Asym_space-MNI152NLin2009cAsym_res-02_T1w.nii.gz \
  --interpolation Linear \
  --transform ../data/tpl-MNI152NLin6Asym_from-MNI152NLin2009cAsym_mode-image_xfm.mat
'''
t1w_mni2009c = ants.ants_apply_transforms(
    input_image=t1w,
    reference_image=ref,
    output=ants.ants_apply_transforms_warped_output_params("tpl-MNI152NLin6Asym_space-MNI152NLin2009cAsym_res-02_T1w.nii.gz"),
    transform=[ants.ants_apply_transforms_transform_file_name_params(xfm)],
    interpolation=ants.ants_apply_transforms_linear_params(),
    dimensionality=3,
    input_image_type=0,
)

Try applying the transformation to the mask as well!

In [ ]:
'''
antsApplyTransforms \
  --dimensionality 3 \
  --input-image-type 0 \
  --input ../data/tpl-MNI152NLin6Asym_res-02_desc-brain_mask.nii.gz \
  --reference-image ../data/tpl-MNI152NLin2009cAsym_res-02_T1w.nii.gz \
  --output tpl-MNI152NLin6Asym_space-MNI152NLin2009cAsym_res-02_T1w.nii.gz \
  --interpolation NearestNeighbor \
  --transform ../data/tpl-MNI152NLin6Asym_from-MNI152NLin2009cAsym_mode-image_xfm.mat
'''

Let's take a look at the outputs!

<!-- Look at folder and at variables -->

In [ ]:
print(f"'t1w_mni2009c': {t1w_mni2009c.output}\n")
print(f"'mask_mni2009c': {???}")

We'll also try to plot this visually!

In [ ]:
from nilearn import plotting

plotting.plot_roi(
    ???,
    bg_img=???
    alpha=0.3,
    cmap='autumn',
    title="Mask over T1w in MNI152NLin2009cAsym space"
)
plotting.show()

Just to double check, lets overlay the mask on the original T1w image to make sure the 
transformation actually occurred!

In [ ]:
plotting.plot_roi(
    ???
    bg_img=???,
    alpha=0.3,
    cmap='autumn',
    title="Transformed mask over original T1w"
)
plotting.show()

The differences are subtle, but yes, a transformation did indeed happen!

### Exercise 3

While there are many brain extraction tools (that we could again use to highlight 
`niwrap`), lets do something different. Instead of using an existing tool, lets use the 
data we transformed from [Example 2](#Example-2) and apply the mask. We'll do all this
exclusively using Python libraries!

In [ ]:
import nibabel as nib 
import numpy as np

In [ ]:
t1w_nii = nib.load(???)
mask_nii = nib.load(???)

plotting.plot_anat(
    ???,
    title="Brain extracted T1w"
)
plotting.show()

While we treated these tasks as separate exercises, you could combine
exercises 2 and 3 together to create a short pipeline, performing
all of your tasks within Python!

### Exercise 4

As an additional exercise / challenge, try to play around with `niwrap` to create your own pipeline or use it on your own data!